In [ ]:
%config Completer.use_jedi = False # To make auto-complete faster

#Reloads imported files automatically
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('../../utils/')

In [ ]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:87% !important; }</style>"))

In [ ]:
import numpy as np

In [ ]:
import matplotlib.pyplot as plt
from matplotlib import colormaps as mplcmaps
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.colors as mplcolors

from matplotlib_param_funcs import set_matplotlib_params,reset_rcParams

set_matplotlib_params()

In [ ]:
import compute_variables as CV
import miscellaneous_functions as MF
import plotting_helpers as PH
import velocity_plot as VP
import map_functions as mapf

In [ ]:
symbol_dict,units_dict = mapf.get_kinematic_symbols_dict(),mapf.get_kinematic_units_dict()

In [ ]:
save_path = "/Users/luismi/Desktop/MRes_UCLan/Other_stuff/Mathematica/anicorr/"

save_path += "python/"
MF.create_dir(save_path)

print("Saving in",save_path)

# 3D

In [ ]:
plt.rcParams["font.size"]=20

In [ ]:
ani_min = -10
cmap = "terrain"

In [ ]:
save_bool = True
# save_bool = False

In [ ]:
# plot

# Create a meshgrid for the range of x and y values
x = np.linspace(ani_min, 1, 400)  # Increase the number for a finer mesh
y = np.linspace(-1, 1, 400)  # Increase the number for a finer mesh
X, Y = np.meshgrid(x, y)

# Calculate Z using the provided function
Z = 90 / np.pi * np.arctan(Y / np.abs(X) * np.sqrt(1 - X))

# Ensure Z ranges from -45 to 45
Z = np.clip(Z, -45, 45)

# Create the plot
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

# Plot the surface
surf = ax.plot_surface(X, Y, Z, cmap=cmap)

# Add a color bar which maps values to colors
cbar = fig.colorbar(surf, shrink=0.5, aspect=5,pad=0.08)
cbar.set_label(symbol_dict["tilt_abs"]+units_dict["tilt_abs"])

# Set labels
ax.set_xlabel(symbol_dict["anisotropy"],labelpad=15)
ax.set_ylabel(symbol_dict["correlation"],labelpad=15)
ax.set_zlabel(symbol_dict["tilt_abs"]+units_dict["tilt_abs"],labelpad=10)

# Set z range
ax.set_zlim(-45, 45)

if save_bool:
    filename = f"3D_terrain_{ani_min}ani_{cmap}Cmap"
    
    for fileformat in [".pdf",".png"]:
        plt.savefig(save_path+filename+fileformat,bbox_inches="tight",dpi=250)
        print(fileformat)
    print("Saved",save_path+filename)
plt.show()

# 2D

In [ ]:
plt.rcParams["font.size"]=25

cbar_ticklabelsize = "small"

In [ ]:
imshow_bool = True
# imshow_bool = False # filled contours

unfilled_contours_bool = True
# unfilled_contours_bool = False

In [ ]:
# vertex_abs = True
vertex_abs = False

In [ ]:
if imshow_bool:
    if vertex_abs:
        cbar_ticks = np.arange(-40,40+10,10)
    else:
        cbar_tick_step = 10 #15
        cbar_ticks = np.arange(-90,90+cbar_tick_step,cbar_tick_step)
        
if not imshow_bool:
    level_step = 0.5
    filled_levels=np.arange(-45,45+level_step,level_step)
        
    cbar_tick_step_over_levels = 20

if unfilled_contours_bool:
    unfilled_cmap = mplcmaps["seismic"]
    
    unfilled_linestyle = "--"
    unfilled_lw = 1.2
    
    if vertex_abs:
        max_unfilled_level = 40
        unfilled_levelstep = 10
    else:
        max_unfilled_level = 90
        unfilled_levelstep = 10
    
    unfilled_levels = np.arange(-max_unfilled_level,max_unfilled_level+unfilled_levelstep,unfilled_levelstep)

In [ ]:
cmap = mplcmaps["jet"]
# cmap = PH.get_truncated_colormap(cmap=mplcmaps["jet"],minval=0.25,maxval=0.75)

joint_cyclic_cmap_bool = False

if joint_cyclic_cmap_bool and not vertex_abs:
    cmap_colors = list(mplcmaps["twilight"](np.linspace(0,0.46,25))) +\
                  list(mplcmaps["turbo"](np.linspace(0,1,50))) +\
                  list(mplcmaps["twilight"](np.linspace(0.57,1,25)))

    cmap = mplcolors.LinearSegmentedColormap.from_list(name="twilight-turbo-joint",colors=cmap_colors)

In [ ]:
ani_min = -2

In [ ]:
aspect = "auto"
# aspect = "equal"

In [ ]:
save_bool = True
# save_bool = False

In [ ]:
# plot

x = np.linspace(ani_min, 1, 2500)
y = np.linspace(-1, 1, 2500)
X, Y = np.meshgrid(x, y)

if vertex_abs: Z = 90 / np.pi * np.arctan(2*Y / np.abs(X) * np.sqrt(1 - X))
else: Z = 90 / np.pi * np.arctan2(2*Y*np.sqrt(1 - X), X)

fig,(ax,cax) = plt.subplots(ncols=2,gridspec_kw={"width_ratios":[1,0.05],"wspace":0.05 if aspect=="auto" else -0.35})

if imshow_bool: surf = ax.imshow(Z,cmap=cmap,origin="lower",extent=[min(x),max(x),min(y),max(y)])
else: surf = ax.contourf(X,Y,Z,cmap=cmap,levels=filled_levels)
    
if True: # colorbar
    plt.colorbar(surf,cax=cax)
    
    cax.set_yticks(
        cbar_ticks if imshow_bool else filled_levels[::cbar_tick_step_over_levels],
    )
    
    cax.set_ylabel((r"$l_\mathrm{v}$" if vertex_abs else r"$\tilde{l}_\mathrm{v}$")+ r" $[^\circ]$",labelpad=10)
    
    if not vertex_abs:
        cax.tick_params(axis="y",labelsize=cbar_ticklabelsize)
    
if unfilled_contours_bool:
    ax.contour(X,Y,Z,levels=unfilled_levels,cmap=unfilled_cmap,linestyles=unfilled_linestyle,linewidths=unfilled_lw)

    norm = plt.Normalize(vmin=min(unfilled_levels), vmax=max(unfilled_levels))

    for lev in zip(unfilled_levels):
        cax.axhline(lev,c=unfilled_cmap(norm(lev)),linestyle=unfilled_linestyle,lw=unfilled_lw)
    
    cax.tick_params(axis='y', which='major', size=0)
    
    if joint_cyclic_cmap_bool and not vertex_abs: # cover the discontinuity
        ax.axhline(y=0,xmin=ani_min,xmax=np.abs(ani_min)/(np.abs(ani_min)+1),color=mplcmaps["twilight"](1),lw=1.5)

if True: # axes
    ax.set_xlabel(r"$\beta_{ij}$")
    ax.set_ylabel(r"$\rho_{ij}$")

    if ani_min < -2 and ani_min>-15:
        xtick_step = 1
        xticks = np.concatenate([np.arange(ani_min,0+xtick_step,xtick_step),[1]])
        ax.set_xticks(xticks)
        ax.set_xticklabels([f"{'–' if t<0 else ''}{abs(MF.check_int(t))}" for t in xticks])

    yticks = ax.get_yticks()
    ax.set_yticks(yticks) #avoid warning
    ax.set_yticklabels([f"{'–' if t<0 else ''}{abs(MF.check_int(t))}" for t in yticks])
    
    xticks = ax.get_xticks()
    ax.set_xticks(xticks) #avoid warning
    ax.set_xticklabels([f"{'–' if t<0 else ''}{abs(MF.check_int(t))}" for t in xticks])

    ax.set_aspect(aspect)
    
if True: # save
    imshow_str = "_imshow" if imshow_bool else f"_contours{level_step}levelStep"
    
    vertex_str = "vertexabs" if vertex_abs else "vertex"
    
    unfilled_contours_str = f"_unfilled{unfilled_cmap.name}{unfilled_levelstep}step" if unfilled_contours_bool else ""
    
    filename = f"{vertex_str}_2D_contours_{ani_min}ani{imshow_str}_{cmap.name}Cmap{unfilled_contours_str}"
    print(filename)

    if save_bool:
        for fileformat in [".pdf",".png"]:
            plt.savefig(save_path+filename+fileformat,bbox_inches="tight",dpi=250)
            print(fileformat)
        print("Saved in",save_path)

    plt.show()

# 1D

In [ ]:
plt.rcParams["font.size"]=23

In [ ]:
corr_fixedVals = [0.5,0.75,1]
colors = ["red","darkorange","gold"]

ani_min = -20

In [ ]:
save_bool = True
# save_bool = False

In [ ]:
# plot

fig,ax=plt.subplots()

ani = np.arange(ani_min,1,0.01)

for corr,color in zip(corr_fixedVals,colors):
    vertex = 1/2*np.arctan(corr*np.sqrt(1-ani)/np.abs(ani))*180/np.pi

    ax.plot(ani,vertex,color=color,label=symbol_dict["correlation"]+r"$=%s$"%str(corr))

ax.set_ylim(0,45)
ax.set_xlim(ani_min,1.5)

xtick_step = 2.5
xticks = np.concatenate([np.arange(ani_min,0+xtick_step,xtick_step),[1]])
ax.set_xticks(xticks)
ax.set_xticklabels([f"{'–' if t<0 else ''}{abs(MF.check_int(t))}" for t in xticks])

ax.set_yticks(np.arange(5,45+5,5))

ax.set_xlabel(symbol_dict["anisotropy"])
ax.set_ylabel(symbol_dict["tilt_abs"]+units_dict["tilt_abs"])

plt.legend(fontsize=20)
plt.grid(axis="both")

if True:
    corr_string = f"{len(corr_fixedVals)}corrVals{min(corr_fixedVals)}to{max(corr_fixedVals)}"
    
    filename = f"1D_variation_{ani_min}ani_{corr_string}_{cmap}Cmap"
    print(filename)

    if save_bool:
        for fileformat in [".pdf",".png"]:
            plt.savefig(save_path+filename+fileformat,bbox_inches="tight",dpi=250)
            print(fileformat)
        print("Saved",save_path)

plt.show()

# Velocity plot

In [ ]:
ani = -1
corr = 1

std_r = 1
std_l = std_r*np.sqrt(1-ani)

covxy = corr*std_r*std_l

tilt_abs = CV.calculate_tilt_from_moments(covxy=covxy,varx=std_r**2,vary=std_l**2,absolute=True)

n_points = 5000

r_vals = np.random.normal(0,std_r,n_points)
l_vals = np.random.normal(0,std_l,n_points)

rotated_x,rotated_y = PH.rotate_by_angle(x=r_vals,y=l_vals,angle=tilt_abs)

In [ ]:
VP.velocity_plot?

In [ ]:
VP.velocity_plot(vx=rotated_x,vy=rotated_y,tilt_abs=True,vel_max=max([max(rotated_x),max(rotated_y)]),tickstep=20)

In [ ]:
fig,ax=plt.subplots()

ax.scatter(rotated_x,rotated_y,s=5)
ax.set_title(r"$l_\mathrm{v}=%.2f ^\circ$"%tilt_abs)
ax.set_aspect("equal")
plt.show()